# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

### Import Libraries

Import necessary libraries

In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [3]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [4]:
def extract_feature(file_name, mfcc):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    result=0
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [5]:
def dataset_options():
    # choose datasets
    ravdess = True
    tess = True
    ravdess_speech = False
    ravdess_song = False
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [6]:
def load_data(test_size=0.2): 
    x,y=[],[]
    
    # feature to extract
    mfcc = True
    
    data = dataset_options()
    paths = []
    if data['ravdess']:
        paths.append("../dataset_features/Actor_*/*.wav")
    elif data['ravdess_speech']:
        paths.append("..\Datasets\RAVDESS\Speech\Actor_*\*.wav")
    elif data['ravdess_song']:
        paths.append("..\Datasets\RAVDESS\Song\Actor_*\*.wav")
        
    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]] #to get emotion according to filename. dictionary emotions is defined above.
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    if data['tess']:
        for file in glob.glob("../TESS_Toronto_emotional_speech_set_data/*AF_*/*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    return {"X":x,"y":y}

In [7]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'ravdess_speech': False, 'ravdess_song': False, 'tess': True}
--- Data loaded. Loading time: 172.2438359260559 seconds ---


In [8]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [9]:
X.shape, y.shape

((8052, 41), (8052, 1))

In [10]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [11]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,emotion
0,0.0,-544.942505,37.640598,-7.628289,6.864829,-13.380677,-2.417591,-12.137745,-8.057130,-14.836030,...,3.623613,-0.405900,0.782749,0.000128,2.697446,0.803102,1.826188,-0.609255,1.844422,angry
1,0.0,-523.922180,36.252670,-19.905519,12.971987,-9.616239,-15.922702,-15.657978,-8.893445,-14.755669,...,2.381103,-6.505177,-6.485410,-2.818655,1.587407,0.349844,1.351712,6.588575,7.494193,happy
2,0.0,-553.291931,40.790520,-9.492151,8.202779,-5.402720,-6.108304,-17.632957,-11.217481,-11.315104,...,0.095194,-0.738742,0.245738,-0.519077,0.679776,0.222292,0.895804,0.158789,0.276128,fear
3,0.0,-393.324371,22.974743,-31.079159,4.853052,-13.357388,-12.805939,-17.525320,-9.771192,-12.389983,...,-2.133341,-2.658438,0.480609,1.072412,2.033163,2.564418,1.277522,0.809124,2.371578,fear
4,0.0,-587.782104,43.909641,-17.656189,13.169994,-7.121557,-13.213776,-12.981407,-11.661613,-12.594745,...,3.169533,-4.859682,-6.721389,-4.410873,1.020319,0.567670,0.182629,5.530495,8.593582,happy


## Shuffling data

In [13]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [14]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")